In [1]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber 
from nipype.interfaces.utility import IdentityInterface, Function    
from nipype.pipeline.engine import Node, Workflow, JoinNode, MapNode
import nipype.interfaces.mrtrix3 as mtx
import nipype.interfaces.fsl as fsl
from nipype.workflows.dmri.mrtrix.diffusion import create_mrtrix_dti_pipeline
from pandas import Series, read_csv, to_numeric
from glob import glob
from os.path import abspath, expanduser, join
from os import chdir, remove, getcwd, makedirs
from shutil import copyfile
from nipype import config, logging
from datetime import date
import AFQ
today = str(date.today())
config.enable_debug_mode()

/usr/local/anaconda3/lib/python3.6/site-packages/nipype/workflows/dmri/mrtrix/group_connectivity.py:16: UserWarning: cmp not installed
  warnings.warn('cmp not installed')


In [10]:
#Set user and path variables

user = expanduser('~')
if user == '/Users/lucindasisk':
    home = join(user, 'Desktop/Milgram/candlab')
    raw_dir = join(home, 'data/mri/bids_recon/shapes')
    proc_dir = join(home, 'analyses/shapes/dwi')
    workflow_dir = join(home, 'analyses/shapes/dwi/tractography_workflow')
    data_dir = join(home, 'analyses/shapes/dwi/tractography_data')
else:
    home = '/gpfs/milgram/project/gee_dylan/candlab'
    raw_dir = join(home, 'data/mri/bids_recon/shapes')
    proc_dir = join(home, 'analyses/shapes/dwi/eddyCUDA_data')
    workflow_dir = join(home, 'analyses/shapes/dwi/tractography_workflow')
    data_dir = join(home, 'analyses/shapes/dwi/tractography_data')
    
# Read in subject subject_list
# subject_info = read_csv(
#     home + '/scripts/shapes/mri/dwi/shapes_dwi_subjList_08.07.2019.txt', sep=' ', header=None)
# subject_list = subject_info[0].tolist()

# Manual subject list
subject_list = ['sub-A200', 'sub-A201']

In [11]:
#Setup Datasink, Infosource, Selectfiles

datasink = Node(DataSink(base_directory = data_dir,
                        substitutions = [('_subject_id_', '')]),
                   name='datasink')

#Set infosource iterables
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

#SelectFiles
template = dict(dti = join(proc_dir,'eddyCUDA_data/3_Eddy_Corrected/{subject_id}/eddy_corrected.nii.gz'),
                bval = join(raw_dir, '{subject_id}/ses-shapesV1/dwi/{subject_id}_ses-shapesV1_dwi.bval'),
                bvec = join(proc_dir,'eddyCUDA_data/3_Eddy_Corrected/{subject_id}/eddy_corrected.eddy_rotated_bvecs'),
                t1 = join(proc_dir, 'preproc_data/2_Transfer/{subject_id}/{subject_id}_ses-shapesV1_T1w_resample_brain.nii.gz'),
                aseg = join(home, 'data/mri/hcp_pipeline_preproc/shapes/{subject_id}/MNINonLinear/aparc.a2009s+aseg.nii.gz'),
                aparc = 
               )

sf = Node(SelectFiles(template, 
                      base_directory = home),
          name = 'sf')
                

### Pull aseg file and extract ROIs

In [22]:
# Reorient topup b0 image to std
reorient1 = Node(fsl.Reorient2Std(output_type='NIFTI_GZ'),
                 name='reorient1')

#Register aseg to T1
register1 = Node(fsl.FLIRT(out_matrix_file='asegtoT1_reorient_reg.mat',
                           rigid2D=True,
                           output_type='NIFTI_GZ'),
                 name='register1')

#Threshold left hippocampus and amygdala (hippocampus = 17, amygdala = 18)
thresh1 = Node(fsl.Threshold(thresh = 17,
                           direction = 'below'),
              name='thresh1')
thresh2 = Node(fsl.Threshold(thresh = 19,
                             direction = 'above',
                             out_file='Left_Hipp_Amyg_ROI.nii.gz'),
              name='thresh2')

#Threshold right hippocampus and amygdala (hippocampus =53, amygdala = 54)
thresh3 = Node(fsl.Threshold(thresh = 52,
                           direction = 'below'),
              name='thresh3')
thresh4 = Node(fsl.Threshold(thresh = 54,
                             direction = 'above',
                             out_file='Right_Hipp_Amyg_ROI.nii.gz'),
              name='thresh4')

#Threshold left OFC (Lh G Orbital Cortex = 11124)
thresh5 = Node(fsl.Threshold(thresh = 11123,
                             direction = 'below'),
              name='thresh5')
thresh6 = Node(fsl.Threshold(thresh = 11124,
                             direction = 'above',
                             out_file='Left_OFC_ROI.nii.gz'),
              name='thresh6')

#Threshold right OFC (Rh G Orbital Cortex = 11124)
thresh7 = Node(fsl.Threshold(thresh = 12123,
                             direction = 'below'),
              name='thresh7')
thresh8 = Node(fsl.Threshold(thresh = 12124,
                             direction = 'above',
                             out_file='Right_OFC_ROI.nii.gz'),
              name='thresh8')

In [23]:
roi_flow = Workflow(name = 'roi_flow')
roi_flow.connect([(infosource, sf, [('subject_id','subject_id')]),
                  (sf, reorient1, [('aseg', 'in_file')]),
                  (reorient1, register1, [('out_file', 'in_file')]),
                  (sf, register1, [('t1', 'reference')]),
                  (register1, datasink, [('out_file', '4_ROIs')]),
                  (register1, thresh1, [('out_file', 'in_file')]),
                  (thresh1, thresh2, [('out_file', 'in_file')]),
                  (register1, thresh3, [('out_file', 'in_file')]),
                  (thresh3, thresh4, [('out_file', 'in_file')]),
                  (thresh2, datasink, [('out_file', '4_ROIs.@par')]),
                  (thresh4, datasink, [('out_file', '4_ROIs.@par.@par')]),
                  (register1, thresh5, [('out_file', 'in_file')]),
                  (thresh5, thresh6, [('out_file', 'in_file')]),
                  (thresh6, datasink, [('out_file', '4_ROIs.@par.@par.@par')]),
                  (register1, thresh7, [('out_file', 'in_file')]),
                  (thresh7, thresh8, [('out_file', 'in_file')]),
                  (thresh8, datasink, [('out_file', '4_ROIs.@par.@par.@par.@par')])
                 ])
                  
roi_flow.base_dir = workflow_dir
roi_flow.write_graph(graph2use = 'flat')
dwi = roi_flow.run('MultiProc', plugin_args={'n_procs': 4})

190914-22:31:55,645 nipype.workflow DEBUG:
	 (roi_flow.infosource, roi_flow.sf): No edge data
190914-22:31:55,646 nipype.workflow DEBUG:
	 (roi_flow.infosource, roi_flow.sf): new edge data: {'connect': [('subject_id', 'subject_id')]}
190914-22:31:55,647 nipype.workflow DEBUG:
	 (roi_flow.sf, roi_flow.reorient1): No edge data
190914-22:31:55,647 nipype.workflow DEBUG:
	 (roi_flow.sf, roi_flow.reorient1): new edge data: {'connect': [('aseg', 'in_file')]}
190914-22:31:55,648 nipype.workflow DEBUG:
	 (roi_flow.reorient1, roi_flow.register1): No edge data
190914-22:31:55,649 nipype.workflow DEBUG:
	 (roi_flow.reorient1, roi_flow.register1): new edge data: {'connect': [('out_file', 'in_file')]}
190914-22:31:55,649 nipype.workflow DEBUG:
	 (roi_flow.sf, roi_flow.register1): No edge data
190914-22:31:55,650 nipype.workflow DEBUG:
	 (roi_flow.sf, roi_flow.register1): new edge data: {'connect': [('t1', 'reference')]}
190914-22:31:55,651 nipype.workflow DEBUG:
	 (roi_flow.register1, roi_flow.data

190914-22:31:57,197 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_sub-A201
190914-22:31:57,198 nipype.workflow DEBUG:
	 PE: expanding iterables ... done
190914-22:31:57,199 nipype.workflow DEBUG:
	 [Node] sf - setting input subject_id = sub-A201
190914-22:31:57,200 nipype.workflow DEBUG:
	 Removed the identity node roi_flow.infosource from the graph.
190914-22:31:57,201 nipype.workflow DEBUG:
	 [Node] sf - setting input subject_id = sub-A200
190914-22:31:57,201 nipype.workflow DEBUG:
	 Removed the identity node roi_flow.infosource from the graph.
190914-22:31:58,690 nipype.workflow DEBUG:
	 Performing depth first search
190914-22:31:58,952 nipype.workflow INFO:
	 Running in parallel.
190914-22:31:58,956 nipype.workflow DEBUG:
	 Progress: 24 jobs, 0/0/2 (done/running/ready), 0/24 (pending_tasks/waiting).
190914-22:31:58,957 nipype.workflow DEBUG:
	 Tasks currently running: 0. Pending: 0.
190914-22:31:58,958 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 job

	 Needed dirs: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/roi_flow/_subject_id_sub-A201/sf/_report
190914-22:32:00,969 nipype.workflow DEBUG:
	 Removing files: 190914-22:32:01,90 nipype.workflow DEBUG:
	 saved results in /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/roi_flow/_subject_id_sub-A201/sf/result_sf.pklz
190914-22:32:01,190 nipype.workflow DEBUG:
	 Needed files: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_data/3_Eddy_Corrected/sub-A200/eddy_corrected.nii.gz;/Users/lucindasisk/Desktop/Milgram/candlab/data/mri/bids_recon/shapes/sub-A200/ses-shapesV1/dwi/sub-A200_ses-shapesV1_dwi.bval;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_data/3_Eddy_Corrected/sub-A200/eddy_corrected.eddy_rotated_bvecs;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_data/2_Transfer/sub-A200/sub-A200_ses-shapesV1_T1w_resample_brain.nii.gz;/Users/lucin

190914-22:32:05,51 nipype.utils DEBUG:
	 Loading pkl: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/roi_flow/_subject_id_sub-A201/reorient1/result_reorient1.pklz
190914-22:32:05,70 nipype.workflow DEBUG:
	 output: out_file
190914-22:32:05,71 nipype.workflow DEBUG:
	 [Node] register1 - setting input in_file = /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/roi_flow/_subject_id_sub-A201/reorient1/aparc.a2009s+aseg_reoriented.nii.gz
190914-22:32:05,201 nipype.workflow DEBUG:
	 [Node] Hashes: [('in_file', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/roi_flow/_subject_id_sub-A201/reorient1/aparc.a2009s+aseg_reoriented.nii.gz', '447b73d1d80c797c26f92cc4b2b62ed9')), ('out_matrix_file', 'asegtoT1_reorient_reg.mat'), ('output_type', 'NIFTI_GZ'), ('reference', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/preproc_data/2_Transfer/sub-A201/sub-A201_ses-shapesV1_T

190914-22:32:07,295 nipype.workflow INFO:
	 [Node] Setting-up "roi_flow.thresh7" in "/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/roi_flow/_subject_id_sub-A201/thresh7".
190914-22:32:07,315 nipype.workflow DEBUG:
	 output: out_file
190914-22:32:07,316 nipype.workflow DEBUG:
	 [Node] thresh5 - setting input in_file = /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/roi_flow/_subject_id_sub-A201/register1/aparc.a2009s+aseg_reoriented_flirt.nii.gz
190914-22:32:07,347 nipype.workflow DEBUG:
	 [Node] Hashes: [('direction', 'below'), ('in_file', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/roi_flow/_subject_id_sub-A201/register1/aparc.a2009s+aseg_reoriented_flirt.nii.gz', 'ef20cffc0a95929474b5aab2b2cc2b5b')), ('output_type', 'NIFTI_GZ'), ('thresh', '12123.0000000000')], 6143592407379d095f02496f6a185caa, /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractograp

...

	 [Node] Writing post-exec report to "/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/roi_flow/_subject_id_sub-A200/thresh1/_report/report.rst"
190914-22:32:39,19 nipype.workflow DEBUG:
	 Aggregate: False
190914-22:32:39,133 nipype.workflow INFO:
	 [Node] Finished "roi_flow.thresh1".
190914-22:32:40,992 nipype.workflow INFO:
	 [Job 21] Completed (roi_flow.thresh1).
190914-22:32:40,994 nipype.workflow DEBUG:
	 Tasks currently running: 1. Pending: 1.
190914-22:32:40,996 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 1 jobs ready. Free memory (GB): 28.60/28.80, Free processors: 3/4.
                     Currently running:
                       * roi_flow.thresh4
190914-22:32:41,50 nipype.workflow DEBUG:
	 Allocating roi_flow.thresh2 ID=22 (0.20GB, 1 threads). Free: 28.40GB, 2 threads.
190914-22:32:41,50 nipype.workflow DEBUG:
	 Setting node inputs
190914-22:32:41,51 nipype.workflow DEBUG:
	 input: in_file
190914-22:32:41,52 nipype.workflow D

	 [Node] Writing pre-exec report to "/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/roi_flow/_subject_id_sub-A200/thresh2/_report/report.rst"
190914-22:32:44,454 nipype.interface DEBUG:
	 in_file_/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/roi_flow/_subject_id_sub-A200/thresh1/aparc.a2009s+aseg_reoriented_flirt_thresh.nii.gz
190914-22:32:44,459 nipype.interface DEBUG:
	 out_file_Left_Hipp_Amyg_ROI.nii.gz
190914-22:32:44,569 nipype.workflow INFO:
	 [Node] Running "thresh2" ("nipype.interfaces.fsl.maths.Threshold"), a CommandLine Interface with command:
fslmaths /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/roi_flow/_subject_id_sub-A200/thresh1/aparc.a2009s+aseg_reoriented_flirt_thresh.nii.gz -uthr 18.0000000000 Left_Hipp_Amyg_ROI.nii.gz
190914-22:32:44,609 nipype.interface DEBUG:
	 in_file_/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workfl

190914-22:32:49,334 nipype.workflow INFO:
	 [Node] Outdated cache found for "roi_flow.datasink".
190914-22:32:49,334 nipype.workflow DEBUG:
	 [Node] Old/new hashes = 64191949527de1d34bde777ba9455c35/8aa31b7fd2702c7bc0539ae35d6318c3
190914-22:32:49,371 nipype.workflow DEBUG:
	           Some dictionary entries had differing values:
            * _outputs: [('4_ROIs', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/roi_flow/_subject_id_sub-A200/register1/aparc.a2009s+aseg_reoriented_flirt.nii.gz', '3b4c90f3cc7d028c7e213215156c6c8d')), ('4_ROIs.@par', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/roi_flow/_subject_id_sub-A200/thresh2/Left_Hipp_Amyg_ROI.nii.gz', '3b9258d6e7379b3da1c840b98262acc7')), ('4_ROIs.@par.@par', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/roi_flow/_subject_id_sub-A200/thresh4/Right_Hipp_Amyg_ROI.nii.gz', 'c40aa86904e63409de5e5a08a1e8fbee')), ('

190914-22:32:51,260 nipype.workflow INFO:
	 [Node] Running "datasink" ("nipype.interfaces.io.DataSink")
190914-22:32:51,265 nipype.interface DEBUG:
	 key: 4_ROIs files: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/roi_flow/_subject_id_sub-A200/register1/aparc.a2009s+aseg_reoriented_flirt.nii.gz
190914-22:32:51,268 nipype.interface DEBUG:
	 sub.str: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_ROIs/_subject_id_sub-A200/aparc.a2009s+aseg_reoriented_flirt.nii.gz -> /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_ROIs/sub-A200/aparc.a2009s+aseg_reoriented_flirt.nii.gz using '_subject_id_' -> ''
190914-22:32:51,272 nipype.interface INFO:
	 sub: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_ROIs/_subject_id_sub-A200/aparc.a2009s+aseg_reoriented_flirt.nii.gz -> /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_RO

	 sub.str: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_ROIs/_subject_id_sub-A200/Left_OFC_ROI.nii.gz -> /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_ROIs/sub-A200/Left_OFC_ROI.nii.gz using '_subject_id_' -> ''
190914-22:32:52,342 nipype.interface INFO:
	 sub: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_ROIs/_subject_id_sub-A200/Left_OFC_ROI.nii.gz -> /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_ROIs/sub-A200/Left_OFC_ROI.nii.gz
190914-22:32:52,367 nipype.interface DEBUG:
	 copyfile: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/roi_flow/_subject_id_sub-A200/thresh6/Left_OFC_ROI.nii.gz /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_ROIs/sub-A200/Left_OFC_ROI.nii.gz
190914-22:32:52,370 nipype.utils DEBUG:
	 /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shap

### Nodes for Diffusion workflow

In [12]:
#Generate binary mask
bet=Node(fsl.BET(frac=0.6,
                mask=True),
        name='bet')

#Generate 5 tissue type (5tt) segmentation using FAST algorithm
seg5tt = Node(mtx.Generate5tt(algorithm = 'fsl',
                             out_file = 'T1s_5tt_segmented.nii.gz'),
             name='seg5tt')

#Estimate response functions for spherical deconvolution using the specified algorithm (Dhollander)
#https://nipype.readthedocs.io/en/latest/interfaces/generated/interfaces.mrtrix3/preprocess.html#responsesd
#https://mrtrix.readthedocs.io/en/latest/constrained_spherical_deconvolution/response_function_estimation.html#response-function-estimation
#Max_sh (lmax variable) determined in shell order from here: https://mrtrix.readthedocs.io/en/3.0_rc2/constrained_spherical_deconvolution/lmax.html
#DWI has 5 shells: 7 b0 volumes, 6 b500 vols, 15 b1000 vols, 15 b2000 bols, 60 b3000 vols
dwiresp = Node(mtx.ResponseSD(algorithm = 'dhollander',
                              max_sh=[0,2,4,4,8],
                              wm_file = 'wm_response.txt',
                              gm_file = 'gm_response.txt',
                              csf_file = 'csf_response.txt'),
              name='dwiresp')

#Estimate fiber orientation distributions from diffusion data sing spherical deconvolution
#https://nipype.readthedocs.io/en/latest/interfaces/generated/interfaces.mrtrix3/reconst.html
#https://mrtrix.readthedocs.io/en/latest/constrained_spherical_deconvolution/multi_shell_multi_tissue_csd.html
#Max SH here determined by tissue type - chose 8,8,8 per forum recommendations
mscsd = Node(mtx.EstimateFOD(algorithm = 'msmt_csd',
                             bval_scale = 'yes',
                            max_sh = [8,8,8]),
            name='mscsd')

#Perform Tractography - ACT using iFOD2 (https://nipype.readthedocs.io/en/latest/interfaces/generated/interfaces.mrtrix3/tracking.html) 
tract = Node(mtx.Tractography(algorithm='iFOD2',
                             n_trials=10000),
            name='tract')

In [16]:
tract_flow = Workflow(name = 'tract_flow')
tract_flow.connect([(infosource, sf, [('subject_id','subject_id')]),
                    (sf, bet, [('t1', 'in_file')]),
                    (sf, seg5tt, [('t1', 'in_file')]),
                    (seg5tt, datasink, [('out_file', '4_tract_Reconstruction')]),
                    (sf, dwiresp, [('dti', 'in_file'),
                                   ('bval','in_bval'),
                                   ('bvec', 'in_bvec')]),
                    (dwiresp, datasink, [('wm_file', '4_tract_Reconstruction.@par'),
                                        ('gm_file', '4_tract_Reconstruction.@par.@par'),
                                        ('csf_file', '4_tract_Reconstruction.@par.@par.@par')]),
                    (sf, mscsd, [('dti', 'in_file'),
                                ('bval', 'in_bval'),
                                ('bvec', 'in_bvec')]),
                    (dwiresp, mscsd, [('wm_file', 'wm_txt'),
                                      ('gm_file', 'gm_txt'),
                                      ('csf_file', 'csf_txt')]),
                    (seg5tt, tract, [('out_file', 'act_file')]),
                    (mscsd, tract, [('wm_odf', 'in_file')]),
                    (mscsd, datasink, [('wm_odf', '4_tract_Reconstruction.@par.@par.@par.@par'),
                                       ('gm_odf', '4_tract_Reconstruction.@par.@par.@par.@par.@par'),
                                       ('csf_odf','4_tract_Reconstruction.@par.@par.@par.@par.@par.@par')]),
                    (sf, tract, [('bval', 'in_bval'),
                                 ('bvec', 'in_bvec')]),
                    (bet, tract, [('mask_file', 'seed_image')]),
                    (tract, datasink, [('out_file', '4_tract_Reconstruction.@par.@par.@par.@par.@par.@par.@par'),
                                     ('out_seeds','4_tract_Reconstruction.@par.@par.@par.@par.@par.@par.@par.@par')])
                   ])
tract_flow.base_dir = workflow_dir
tract_flow.write_graph(graph2use = 'flat')
dwi = tract_flow.run('MultiProc', plugin_args={'n_procs': 4})

190903-22:15:50,204 nipype.workflow DEBUG:
	 (tract_flow.infosource, tract_flow.sf): No edge data
190903-22:15:50,205 nipype.workflow DEBUG:
	 (tract_flow.infosource, tract_flow.sf): new edge data: {'connect': [('subject_id', 'subject_id')]}
190903-22:15:50,207 nipype.workflow DEBUG:
	 (tract_flow.sf, tract_flow.bet): No edge data
190903-22:15:50,207 nipype.workflow DEBUG:
	 (tract_flow.sf, tract_flow.bet): new edge data: {'connect': [('t1', 'in_file')]}
190903-22:15:50,208 nipype.workflow DEBUG:
	 (tract_flow.sf, tract_flow.seg5tt): No edge data
190903-22:15:50,209 nipype.workflow DEBUG:
	 (tract_flow.sf, tract_flow.seg5tt): new edge data: {'connect': [('t1', 'in_file')]}
190903-22:15:50,209 nipype.workflow DEBUG:
	 (tract_flow.seg5tt, tract_flow.datasink): No edge data
190903-22:15:50,210 nipype.workflow DEBUG:
	 (tract_flow.seg5tt, tract_flow.datasink): new edge data: {'connect': [('out_file', '4_tract_Reconstruction')]}
190903-22:15:50,210 nipype.workflow DEBUG:
	 (tract_flow.sf, t

190903-22:15:53,650 nipype.workflow DEBUG:
	 Allocating tract_flow.sf ID=0 (0.20GB, 1 threads). Free: 28.60GB, 3 threads.
190903-22:15:53,651 nipype.workflow DEBUG:
	 Setting node inputs
190903-22:15:53,777 nipype.workflow DEBUG:
	 [Node] Hashes: [('base_directory', '/Users/lucindasisk/Desktop/Milgram/candlab'), ('force_lists', False), ('raise_on_empty', True), ('sort_filelist', True), ('subject_id', 'sub-A201')], 1ba667db9b42399b10348726f58f915e, /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A201/sf/_0x1ba667db9b42399b10348726f58f915e.json, ['/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A201/sf/_0x1ba667db9b42399b10348726f58f915e.json']
190903-22:15:53,778 nipype.workflow DEBUG:
	 [Node] Up-to-date cache found for "tract_flow.sf".
190903-22:15:53,779 nipype.workflow DEBUG:
	 Checking hash "tract_flow.sf" locally: cached=True, updated=True.
190903-22:15:5

190903-22:15:58,206 nipype.workflow DEBUG:
	 Checking hash "tract_flow.seg5tt" locally: cached=True, updated=True.
190903-22:15:58,207 nipype.workflow DEBUG:
	 Skipping cached node tract_flow.seg5tt with ID 3.
190903-22:15:58,208 nipype.workflow INFO:
	 [Job 3] Cached (tract_flow.seg5tt).
190903-22:15:58,210 nipype.workflow DEBUG:
	 Allocating tract_flow.bet ID=4 (0.20GB, 1 threads). Free: 28.20GB, 1 threads.
190903-22:15:58,211 nipype.workflow DEBUG:
	 Setting node inputs
190903-22:15:58,212 nipype.workflow DEBUG:
	 input: in_file
190903-22:15:58,213 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A201/sf/result_sf.pklz
190903-22:15:58,214 nipype.utils DEBUG:
	 Loading pkl: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A201/sf/result_sf.pklz
190903-22:15:58,239 nipype.workflow DEBUG:
	 output: t1
190903-22:15:58,239 ni

190903-22:15:59,695 nipype.workflow DEBUG:
	 input: in_bvec
190903-22:15:59,695 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A201/sf/result_sf.pklz
190903-22:15:59,696 nipype.utils DEBUG:
	 Loading pkl: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A201/sf/result_sf.pklz
190903-22:15:59,722 nipype.workflow DEBUG:
	 output: bvec
190903-22:15:59,723 nipype.workflow DEBUG:
	 [Node] mscsd - setting input in_bvec = /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/eddyCUDA_data/3_Eddy_Corrected/sub-A201/eddy_corrected.eddy_rotated_bvecs
190903-22:15:59,723 nipype.workflow DEBUG:
	 input: wm_txt
190903-22:15:59,724 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A201/dwiresp/result_dwiresp.pklz
190903-22:15:5

190903-22:16:00,227 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/dwiresp/result_dwiresp.pklz
190903-22:16:00,227 nipype.utils DEBUG:
	 Loading pkl: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/dwiresp/result_dwiresp.pklz
190903-22:16:00,249 nipype.workflow DEBUG:
	 output: gm_file
190903-22:16:00,250 nipype.workflow DEBUG:
	 [Node] mscsd - setting input gm_txt = /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/dwiresp/gm_response.txt
190903-22:16:00,250 nipype.workflow DEBUG:
	 input: csf_txt
190903-22:16:00,251 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/dwiresp/result_dwiresp.pklz
190903-22:16:00,251 nipype.utils DEBUG:
	 Loadi

...

190903-22:17:47,948 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/mscsd/result_mscsd.pklz
190903-22:17:47,949 nipype.utils DEBUG:
	 Loading pkl: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/mscsd/result_mscsd.pklz
190903-22:17:47,972 nipype.workflow DEBUG:
	 output: csf_odf
190903-22:17:47,972 nipype.workflow DEBUG:
	 [Node] datasink - setting input 4_tract_Reconstruction.@par.@par.@par.@par.@par.@par = /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/mscsd/csf.mif
190903-22:17:47,973 nipype.workflow DEBUG:
	 input: 4_tract_Reconstruction.@par.@par.@par.@par.@par.@par.@par
190903-22:17:47,973 nipype.workflow DEBUG:
	 results file: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub

190903-22:17:48,353 nipype.workflow INFO:
	 [Node] Setting-up "tract_flow.datasink" in "/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/datasink".
190903-22:17:48,399 nipype.workflow DEBUG:
	 [Node] Hashes: [('_outputs', [('4_tract_Reconstruction', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/seg5tt/T1s_5tt_segmented.nii.gz', 'a2a50d0b7a3fd3ac0f406b9a39ea9e0c')), ('4_tract_Reconstruction.@par', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/dwiresp/wm_response.txt', '42bb40b3aa19ce0834d5dc27c8e18081')), ('4_tract_Reconstruction.@par.@par', ('/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A200/dwiresp/gm_response.txt', '2cc9afe925ee3aa371eb4641cd83357d')), ('4_tract_Reconstruction.@par.@par.@par', ('/Users/lucin

	 File: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A201/csf_response.txt already exists,51479c7a5ee2391a2a13d4b25f2eae2e, copy:1
190903-22:17:54,286 nipype.utils DEBUG:
	 File: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A201/csf_response.txt already exists, not overwriting, copy:1
190903-22:17:54,290 nipype.utils DEBUG:
	 /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A201/csf_response.txt
190903-22:17:54,319 nipype.utils DEBUG:
	 File: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A201/csf_response.txt already exists,51479c7a5ee2391a2a13d4b25f2eae2e, copy:1
190903-22:17:54,349 nipype.utils DEBUG:
	 File: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A201/csf_response.txt alre

	 sub.str: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/_subject_id_sub-A201/gm.mif -> /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A201/gm.mif using '_subject_id_' -> ''
190903-22:51:03,854 nipype.interface INFO:
	 sub: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/_subject_id_sub-A201/gm.mif -> /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A201/gm.mif
190903-22:51:03,880 nipype.interface DEBUG:
	 copyfile: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_workflow/tract_flow/_subject_id_sub-A201/mscsd/gm.mif /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A201/gm.mif
190903-22:51:03,883 nipype.utils DEBUG:
	 /Users/lucindasisk/Desktop/Milgram/candlab/analyses/sh

	 Needed files: /Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A201/T1s_5tt_segmented.nii.gz;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A201/wm_response.txt;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A201/gm_response.txt;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A201/csf_response.txt;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A201/wm.mif;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A201/gm.mif;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstruction/sub-A201/csf.mif;/Users/lucindasisk/Desktop/Milgram/candlab/analyses/shapes/dwi/tractography_data/4_tract_Reconstr

In [ ]:
# (mscsd, datasink, [('wm_odf', '4_tract_Reconstruction.@par.@par.@par.@par'),
#                                        ('gm_odf', '4_tract_Reconstruction.@par.@par.@par.@par.@par'),
#                                        ('csf_odf','4_tract_Reconstruction.@par.@par.@par.@par.@par.@par')]),


# dwi_flow = Workflow(name = 'dwi_flow')
# dwi_flow.connect([(infosource, sf, [('subject_id','subject_id')]),
#                   (sf, mrconv, [('dti', 'in_file')]),
#                   (mrconv, dwi_res, [('out_file', 'in_file')]),
#                   (sf, dwi_res, [('bval', 'in_bval'),
#                                  ('bvec', 'in_bvec')]),
#                   (dwi_res, datasink, [('csf_file', '4_DWI_Reconstruction.@par'),
#                                       ('wm_file', '4_DWI_Reconstruction.@par.@par'),
#                                       ('gm_file', '4_DWI_Reconstruction.@par.@par.@par')]),
#                   (sf, ms_csd, [('dti','in_file'),
#                                ('bval','in_bval'),
#                                ('bvec','in_bvec'),
#                                ('mask', 'mask_file')]),
#                   (dwi_res, ms_csd, [('wm_file', 'wm_txt'),
#                                      ('gm_file', 'gm_txt'),
#                                      ('csf_file', 'csf_txt')]),
#                   (ms_csd, datasink, [('wm_odf', '4_DWI_Reconstruction.@par.@par.@par.@par'),
#                                      ('gm_odf','4_DWI_Reconstruction.@par.@par.@par.@par.@par'),
#                                      ('csf_odf', '4_DWI_Reconstruction.@par.@par.@par.@par.@par.@par')]) 
#                  ])
# dwi_flow.base_dir = workflow_dir
# dwi_flow.write_graph(graph2use = 'flat')
# dwi = dwi_flow.run('MultiProc', plugin_args={'n_procs': 4})